In [2]:
from config.Experiment_Config import ExperimentConfig
from exp.trainer import Trainer

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
config = ExperimentConfig.get_default_config()
config.model_settings.embedding_type = 'XLM_roberta_large'
config.training_settings.num_epochs = 1000
config.training_settings.batch_size = 32
config.tokenizer_settings.max_length = 256
config.model_settings.weight_init = 'kaiming_normal'
config.data_settings.imbalanced_strategy = 'weighted_sampler'
config.data_settings.alpha = 0.1
config.model_settings.activation = 'gelu'
config.model_settings.fine_tune_embedding = True
config.training_settings.early_stopping_patience = 100
config.training_settings.task_type = 'Multi'
config.model_settings.num_layers = 12
config.model_settings.custom_hidden_dims = [1024, 1024, 512, 512, 256, 256, 128, 128, 64, 64, 64, 64]
config.model_settings.use_res_net = True
config.model_settings.use_attention = False
config.training_settings.gradient_clip = 1.0
config.training_settings.continue_training = True
config.training_settings.learning_rate = 2e-5
config.model_settings.fine_tune_lr = 5e-5
config.model_settings.use_layer_norm = True
config.model_settings.fine_tune_embedding = True
config.model_settings.attention_temperature = 1.0
config.model_settings.attention_positions = ['embedding', 'inter_lstm']
config.model_settings.use_attention = True
config.model_settings.fine_tune_loading_strategies = ['periodic', 'plateau']
config.model_settings.fine_tune_mode = 'gradual'
config.model_settings.num_frozen_layers = 12
config.model_settings.fine_tune_reload_freq = 20
config.model_settings.dropout_rate = 0.2
config.model_settings.gradual_unfreeze_epochs = 50
config.model_settings.bidirectional = True


In [ ]:
trainer = Trainer(
    config=config
)

# Train model
metrics_history = trainer.train()

Token indices sequence length is longer than the specified maximum sequence length for this model (1024 > 512). Running this sequence through the model will result in indexing errors


🔄 Gradual fine-tuning initialized (starting with last layer)
📊 Trainable parameters: 7,087,872 / 109,482,240 (6.5%)
🔄 Periodic reload triggered at epoch 0
✅ Loaded best model weights

EXPERIMENT CONFIGURATION

📊 Experiment Name: BL_XL_L12H256M256_W2e-5FT5e-5CE-GeLU_WS_ATN-EI_LD-PL20_ARNTG
📁 Model Directory: saved_models\Multi\question\BL_XL_L12H256M256_W2e-5FT5e-5CE-GeLU_WS_ATN-EI_LD-PL20_ARNTG
📈 Results Directory: results\Multi\question\BL_XL_L12H256M256_W2e-5FT5e-5CE-GeLU_WS_ATN-EI_LD-PL20_ARNTG

🔧 Model Configuration:
- Model Type: BiLSTM
- Embedding Type: XLM_roberta_large
- Architecture:
  • Hidden Dimensions: [1024, 1024, 512, 512, 256, 256, 128, 128, 64, 64, 64, 64]
  • Number of Layers: 12
  • Bidirectional: True
  • Dropout Rate: 0.2

🎯 Attention Configuration:
- Use Attention: True
  • Number of Heads: 8
  • Attention Positions: ['embedding', 'inter_lstm']
  • Attention Dropout: 0.1
  • Temperature: 1.0

🏗️ Architecture Features:
- Residual Connections: True
  • Residual Drop

Training:   1%|          | 1/127 [00:00<01:44,  1.21it/s]


📊 Training Dynamics Monitor (Epoch 1):
⚠️ Vanishing gradient in embedding_model.encoder.layer.11.attention.self.key.bias: 0.0000
LSTM - Mean Grad: 6.85e-05, Max Norm: 5.40e-02
FC - Mean Grad: 3.60e-04, Max Norm: 5.36e-02
ATTENTION - Mean Grad: 1.09e-04, Max Norm: 7.71e-02
Activations first_layer:
  Mean: 0.7749, Std: 1.0001
Activations last_layer:
  Mean: 0.0263, Std: 0.0378
⚠️ Low activation values in last_layer
Outputs - Mean: 0.0263, Std: 0.0378


Training: 100%|██████████| 127/127 [00:27<00:00,  4.57it/s]


...


📊 Training Dynamics Monitor (Epoch 3):
⚠️ Vanishing gradient in embedding_model.encoder.layer.11.attention.self.key.bias: 0.0000
LSTM - Mean Grad: 1.65e-04, Max Norm: 1.42e-01
FC - Mean Grad: 6.01e-04, Max Norm: 1.22e-01
ATTENTION - Mean Grad: 2.97e-04, Max Norm: 2.48e-01
Activations first_layer:
  Mean: 0.7723, Std: 1.0014
Activations last_layer:
  Mean: 0.0263, Std: 0.0893
⚠️ Low activation values in last_layer
Outputs - Mean: 0.0263, Std: 0.0893


Training: 100%|██████████| 127/127 [00:30<00:00,  4.15it/s]



📊 Train Metrics (Epoch 3)
═════════════════════════════════════════════════════════════════
│ Accuracy     │ 0.2790       │
─────────────────────────────────────────────────────────────────
│ F1 Macro     │ 0.1890       │ F1 Micro     │ 0.2790       │
─────────────────────────────────────────────────────────────────
│ Prec Macro   │ 0.2202       │ Prec Micro   │ 0.2790       │
─────────────────────────────────────────────────────────────────
│ Recall Macro │ 0.2828       │ Recall Micro │ 0.2790       │
═════════════════════════════════════════════════════════════════
✅ Loaded latest fine-tuned embedding for val evaluation


Evaluating (val): 100%|██████████| 15/15 [00:02<00:00,  6.01it/s]



📊 Val Metrics (Epoch 3)
═════════════════════════════════════════════════════════════════
│ Accuracy     │ 0.4146       │
─────────────────────────────────────────────────────────────────
│ F1 Macro     │ 0.1067       │ F1 Micro     │ 0.4146       │
─────────────────────────────────────────────────────────────────
│ Prec Macro   │ 0.1458       │ Prec Micro   │ 0.4146       │
─────────────────────────────────────────────────────────────────
│ Recall Macro │ 0.1754       │ Recall Micro │ 0.4146       │
═════════════════════════════════════════════════════════════════
💾 Saved latest training plots for epoch 3
💾 Saved latest validation plots for epoch 3
💾 Saved latest metrics at epoch 3
💾 Saved latest fine-tuned embedding at epoch 3
💾 Saved latest model at epoch 3
🏆 Saved best model at epoch 3
🏆 Saved best training plots for epoch 3
🏆 Saved best validation plots for epoch 3
🏆 Saved best metrics at epoch 3
🏆 Saved best fine-tuned embedding at epoch 3
